<a href="https://colab.research.google.com/github/daniel-nuno/time_series_O2024_MAF3074N/blob/main/evaluacion_pronosticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

<br>
<br>
Clase: Series de tiempo
<br>
Actividad: Evaluación y Pronósticos
<br>
<br>

* * *

Docente: Daniel Nuño <br>
Fecha: 7 de octubre 2024 <br>

* * *

<br>
<br>

# Evaluación de pronósticos

Es importante evaluar la precisión de las previsiones utilizando previsiones auténticas. Por consiguiente, el tamaño de los residuos no es una indicación fiable de la magnitud de los errores de previsión reales. La precisión de las previsiones sólo puede determinarse teniendo en cuenta el rendimiento de un modelo con datos nuevos que no se utilizaron al ajustarlo.

Hay que tener en cuenta los siguientes puntos.

* Un modelo que se ajuste bien a los datos de entrenamiento no necesariamente pronosticará bien.
* Siempre se puede obtener un ajuste perfecto utilizando un modelo con suficientes parámetros.
* Ajustar demasiado un modelo a los datos es tan malo como no identificar un patrón sistemático en los datos.
* Algunas referencias describen el conjunto de prueba como el «conjunto de retención» porque estos datos se «retienen» de los datos utilizados para el ajuste.

Otras referencias llaman al conjunto de entrenamiento «datos dentro de la muestra» y al conjunto de prueba «datos fuera de la muestra». En este libro preferimos utilizar «datos de entrenamiento» y «datos de prueba».

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy as sp
import yfinance as yf

In [3]:
data = pd.read_excel("/content/drive/MyDrive/series_tiempo/Assets/remesas.xlsx")
data.index = pd.to_datetime(data['Fecha'])
del data['Fecha']
del data['Promedio Diario']
data = pd.Series(data = data['Remesas Totales'], index=data.index)

In [4]:
data.tail()

,Remesas Totales
Fecha,
2024-03-01,5014.5816
2024-04-01,5418.8526
2024-05-01,5618.1155
2024-06-01,6206.8793
2024-07-01,5613.6041


In [5]:
training_mask = data.index > '2023-01-01'

In [6]:
test = data[training_mask]
train = data[~training_mask]
horizon = len(test)

In [7]:
test.head()

,Remesas Totales
Fecha,
2023-02-01,4346.1517
2023-03-01,5189.3920
2023-04-01,5006.8015
2023-05-01,5675.8614
2023-06-01,5584.4913


In [8]:
pip install statsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.1 MB/s eta 0:00:00


In [10]:
from statsforecast.models import SeasonalNaive

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [11]:
model_sn = SeasonalNaive(season_length=12)
model_sn = model_sn.fit(y=train.values)
y_hat_dict = model_sn.predict(h=horizon)

y_hat_dict['mean']

array([3928.4852, 4712.0937, 4749.1098, 5173.7153, 5150.2223, 5342.837 ,
       5160.1441, 5078.5862, 5404.6702, 4853.746 , 5379.1316, 4434.7719,
       3928.4852, 4712.0937, 4749.1098, 5173.7153, 5150.2223, 5342.837 ])

In [13]:
model_sn.predict_in_sample()["fitted"]

array([      nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
        254.5724,  248.0605,  287.3645,  298.9135,  357.7396,  352.3695,
        342.8993,  362.2739,  314.1492,  324.5033,  255.5922,  274.2883,
        313.5091,  281.9257,  337.3185,  393.3877,  413.542 ,  365.2451,
        373.6705,  386.0666,  339.2533,  348.901 ,  315.6016,  355.2609,
        338.6477,  331.5918,  381.9072,  425.5386,  486.6894,  453.5596,
        441.6863,  428.9022,  431.4678,  421.6681,  343.4307,  379.755 ,
        382.5317,  366.3982,  427.1949,  439.995 ,  520.3739,  503.4876,
        494.3256,  486.607 ,  476.2905,  454.6739,  460.6924,  614.271 ,
        399.6423,  388.8511,  464.8977,  469.1828,  571.562 ,  521.9225,
        506.6642,  532.0997,  490.4829,  474.5069,  502.0073,  587.7353,
        456.2456,  447.187 ,  494.4502,  498.8308,  590.7496,  541.5775,
        557.598 ,  608.0612,  568.5493,  559.5235, 

In [14]:
residuals_train = train.values[12:] - model_sn.predict_in_sample()['fitted'][12:]

In [37]:
residuals_train = train.values[12:] - model_sn.predict_in_sample()['fitted'][12:]
residuals_test = test.values - y_hat_dict['mean']

In [16]:
residuals_train = train.values[12:] - model_sn.predict_in_sample()['fitted'][12:]
residuals_test = test.values - y_hat_dict['mean']
rmse_trian = np.sqrt(np.mean(residuals_train**2))
rmse_test = np.sqrt(np.mean(residuals_test**2))

In [17]:
tabla_modelos = pd.DataFrame(columns=['Modelo', 'RMSE Train', 'RMSE Test', 'Parametros'], data = [['Seasonal Naive', rmse_trian, rmse_test, 0]])

In [18]:
tabla_modelos

,Modelo,RMSE Train,RMSE Test,Parametros
0,Seasonal Naive,316.560422,467.073,0


## RMSE

En machine learning, el RMSE (Root Mean Squared Error o Raíz del Error Cuadrático Medio) es una métrica comúnmente utilizada para evaluar el rendimiento de un modelo en problemas de regresión. Estas son algunas de sus principales ventajas:

* Penaliza los errores grandes: Como el RMSE eleva al cuadrado las diferencias
entre las predicciones y los valores reales antes de promediar, los errores más grandes tienen un impacto mayor en la métrica final. Esto es útil cuando deseas que el modelo sea particularmente sensible a los errores grandes.
* Fácil interpretación: El RMSE está en la misma escala que la variable de salida, lo que facilita su interpretación. Por ejemplo, si estás prediciendo precios de casas en miles de dólares, un RMSE de 5 significa que el error promedio es de 5,000 dólares.
* Sensibilidad a cambios pequeños: Debido a la elevación al cuadrado, el RMSE puede detectar cambios pequeños en las predicciones, lo que lo hace adecuado cuando se busca mejorar el modelo finamente.
* Comparación entre modelos: RMSE es una métrica estándar ampliamente utilizada, lo que facilita la comparación del rendimiento de diferentes modelos de regresión en un conjunto de datos específico.
* Diferenciable: En el contexto del entrenamiento de modelos basados en métodos de optimización como el gradiente descendente, el RMSE es una función diferenciable, lo que permite calcular su gradiente y optimizar los pesos del modelo de manera eficiente.

## R-cuadrada

La principal diferencia entre RMSE (Root Mean Squared Error) y R-cuadrada (Coeficiente de Determinación) radica en el tipo de información que ofrecen y en cómo se interpretan en el contexto de los modelos de regresión.

1. RMSE (Root Mean Squared Error)
Métrica de error: El RMSE mide directamente el error promedio de las predicciones de un modelo. Cuantifica la magnitud del error en las predicciones del modelo, representando la distancia media entre los valores observados y los valores predichos.
Interpretación: Se interpreta en las mismas unidades que la variable dependiente o de salida. Un RMSE más bajo indica un modelo más preciso, pero no proporciona un sentido claro de qué tan bien el modelo está explicando la variabilidad de los datos.
Escala: Depende de las unidades de la variable dependiente. Esto significa que puede ser difícil comparar el RMSE entre modelos aplicados a diferentes conjuntos de datos o variables de diferentes escalas.
2. R-cuadrada (Coeficiente de Determinación)
* Métrica de ajuste: mide qué proporción de la variabilidad total en los datos de salida está explicada por el modelo. No mide el error en sí, sino qué tan bien las predicciones del modelo capturan las tendencias generales en los datos.
* Interpretación: va de 0 a 1 (o puede ser negativo si el modelo es peor que una línea de base), donde:
*   =1 significa que el modelo predice perfectamente todos los puntos.

*  =0 significa que el modelo no predice mejor que la media de los datos (es decir, no explica nada de la variabilidad).
*   < 0 significa que el modelo es peor que simplemente usar la media como predicción.

* Escala relativa: Como es una proporción, es más fácil de interpretar y comparar entre diferentes modelos o conjuntos de datos, ya que no está influenciado por las unidades de la variable dependiente.

3. Comparación clave:
* RMSE se centra en la magnitud del error en las predicciones. Si el RMSE es pequeño, significa que el modelo predice bien en términos absolutos.
* R-cuadrada se enfoca en la proporción de la variabilidad de los datos que el modelo puede explicar. Un alto R-cuadrada significa que el modelo está capturando bien las tendencias generales de los datos, pero no garantiza que los errores individuales sean pequeños (ahí es donde entra el RMSE).

**En resumen:**

* RMSE te dice "cuánto se están equivocando las predicciones en promedio.
* R-cuadrada "te dice "qué porcentaje de la variabilidad en los datos está siendo explicado por el modelo."

Ambas métricas son útiles, pero proporcionan información diferente. Es común usar ambas para evaluar modelos de regresión.

$$
R_{2} = 1 - \frac{SS_{res}}{SS_{tot}}
$$

$ SS_{res} $: es la suma de los cuadrados de los residuos (pronósticos) (también llamado error residual)

$ SS_{tot} $: es la suma total de los cuadrados (media) (también llamado total de la variabilidad):

## Decomposición

In [20]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(train, model='additive')
print(result.trend)
print(result.seasonal)
print(result.resid)
print(result.observed)

Fecha
1995-01-01   NaN
1995-02-01   NaN
1995-03-01   NaN
1995-04-01   NaN
1995-05-01   NaN
              ..
2022-09-01   NaN
2022-10-01   NaN
2022-11-01   NaN
2022-12-01   NaN
2023-01-01   NaN
Name: trend, Length: 337, dtype: float64
Fecha
1995-01-01   -289.141684
1995-02-01   -238.510377
1995-03-01     88.183928
1995-04-01     -4.544499
1995-05-01    229.499669
                 ...    
2022-09-01     -4.680994
2022-10-01     52.169448
2022-11-01   -122.082215
2022-12-01    -33.325633
2023-01-01   -289.141684
Name: seasonal, Length: 337, dtype: float64
Fecha
1995-01-01   NaN
1995-02-01   NaN
1995-03-01   NaN
1995-04-01   NaN
1995-05-01   NaN
              ..
2022-09-01   NaN
2022-10-01   NaN
2022-11-01   NaN
2022-12-01   NaN
2023-01-01   NaN
Name: resid, Length: 337, dtype: float64
Fecha
1995-01-01     254.5724
1995-02-01     248.0605
1995-03-01     287.3645
1995-04-01     298.9135
1995-05-01     357.7396
                ...    
2022-09-01    5078.5862
2022-10-01    5404.6702
2022-11-0

In [25]:
tendencia = result.trend

In [33]:
# prompt: remove nan observations from tendencia

tendencia = tendencia.dropna()


In [34]:
tendencia.tail()

,trend
Fecha,
2022-03-01,4768.348842
2022-04-01,4818.063717
2022-05-01,4847.856054
2022-06-01,4880.179879
2022-07-01,4926.472562


In [26]:
from statsforecast.models import RandomWalkWithDrift

In [113]:
model__stl_drift = RandomWalkWithDrift()
model__stl_drift = model__stl_drift.fit(y=tendencia.values)
y_hat_dict_trend = model__stl_drift.predict(h=horizon)

y_hat_dict_trend['mean']
len(model__stl_drift.predict_in_sample()["fitted"])

325

In [114]:
season = result.seasonal

In [115]:
model_sn = SeasonalNaive(season_length=12)
model_sn = model_sn.fit(y=season.values)
y_hat_dict_season = model_sn.predict(h=horizon)

y_hat_dict_season['mean']
season_train_predict = model_sn.predict_in_sample()["fitted"]

In [116]:
season_train_predict = season_train_predict[~np.isnan(season_train_predict)]

In [117]:
len(season_train_predict)

325

In [118]:
predict_stl = model__stl_drift.predict_in_sample()["fitted"] + season_train_predict

In [119]:
y_hat = y_hat_dict_trend['mean'] + y_hat_dict_season['mean']

In [120]:
predict_stl[0] = train.values[12]

ValueError: setting an array element with a sequence.

In [67]:
residuals_train = train.values[12:] - predict_stl
residuals_test = test.values - y_hat
rmse_train = np.sqrt(np.mean(residuals_train**2))
rmse_test = np.sqrt(np.mean(residuals_test**2))

In [68]:
rmse_test, rmse_train

(438.17875234271077, 215.93778433850665)

In [109]:
tabla_modelos = pd.concat([tabla_modelos,
           pd.DataFrame(columns=['Modelo', 'RMSE Train', 'RMSE Test', 'Parametros'],
                        data = [['STL', rmse_train, rmse_test, 0]])])

## Regresión

In [74]:
stock = 'MXN=X'
ticker = yf.Ticker(stock)
usdmxn = ticker.history(start= '1993-01-01', end= '2024-07-31', interval='1mo')['Close']

usdmxn.tail()

,Close
Date,
2024-03-01 00:00:00+00:00,16.546101
2024-04-01 00:00:00+01:00,17.011560
2024-05-01 00:00:00+01:00,17.005819
2024-06-01 00:00:00+01:00,18.323500
2024-07-01 00:00:00+01:00,18.752800


In [77]:
data = data["2003":]

In [76]:
usdmxn.index = usdmxn.index.strftime('%Y-%m')
usdmxn.index = pd.to_datetime(usdmxn.index)
usdmxn.head()

,Close
Date,
2003-12-01,11.191
2004-01-01,11.048
2004-02-01,11.055
2004-03-01,11.105
2004-04-01,11.352


In [78]:
data = pd.DataFrame(data.values, columns=['Y'], index=data.index)
data.head()

,Y
Fecha,
2003-01-01,1051.2547
2003-02-01,979.7597
2003-03-01,1139.1124
2003-04-01,1202.5097
2003-05-01,1350.9669


In [79]:
data = data.join(usdmxn)

In [80]:
data.head(24)

,Y,Close
Fecha,,
2003-01-01,1051.2547,NaN
2003-02-01,979.7597,NaN
2003-03-01,1139.1124,NaN
2003-04-01,1202.5097,NaN
2003-05-01,1350.9669,NaN
2003-06-01,1351.1707,NaN
2003-07-01,1361.3620,NaN
2003-08-01,1401.2462,NaN
2003-09-01,1365.5085,NaN


In [81]:
data['mes'] = data.index.month_name()
data.head()

,Y,Close,mes
Fecha,,,
2003-01-01,1051.2547,NaN,January
2003-02-01,979.7597,NaN,February
2003-03-01,1139.1124,NaN,March
2003-04-01,1202.5097,NaN,April
2003-05-01,1350.9669,NaN,May


In [82]:
data = pd.get_dummies(data, columns=['mes'],
                      prefix="", prefix_sep="",
                      drop_first=True, dtype=float)

In [84]:
fecha_2008 = pd.Series(data = [1],
                       index=pd.to_datetime(["2008-11-01"]),
                       name='2008_outlier')
fecha_2020 = pd.Series(data = [1],
                       index=pd.to_datetime(["2020-03-01"]),
                       name='2020_outlier')

In [85]:
data = data.join(fecha_2008).fillna(0)

In [86]:
data = data.join(fecha_2020).fillna(0)

In [87]:
data['lag1'] = data['Y'].shift(1)
data['lag2'] = data['Y'].shift(2)
data['lag3'] = data['Y'].shift(3)
data['lag4'] = data['Y'].shift(4)
data['lag5'] = data['Y'].shift(5)
data['lag6'] = data['Y'].shift(6)
data['lag7'] = data['Y'].shift(7)
data['lag8'] = data['Y'].shift(8)
data['lag9'] = data['Y'].shift(9)
data['lag10'] = data['Y'].shift(10)
data['lag11'] = data['Y'].shift(11)
data['lag12'] = data['Y'].shift(12)

In [88]:
data.columns

Index(['Y', 'Close', 'August', 'December', 'February', 'January', 'July',
       'June', 'March', 'May', 'November', 'October', 'September',
       '2008_outlier', '2020_outlier', 'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
       'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11', 'lag12'],
      dtype='object')

In [90]:
training_mask = data.index > '2023-01-01'

In [92]:
test = data[training_mask]
train = data[~training_mask]
horizon = len(test)

In [95]:
horizon

18

In [99]:
train = train["2004":]

In [131]:
from sklearn.linear_model import LinearRegression

In [132]:
y_train_rl = LinearRegression().fit(train.drop(columns=['Y']),
                                train['Y']).predict(train.drop(
                                            columns=['Y']))

In [133]:
y_test_rl = LinearRegression().fit(train.drop(columns=['Y']),
                                train['Y']).predict(test.drop(
                                            columns=['Y']))

In [134]:
residuals_test = test['Y'].values - y_test_rl
residuals_train = train['Y'].values - y_train_rl
rmse_train = np.sqrt(np.mean(residuals_train**2))
rmse_test = np.sqrt(np.mean(residuals_test**2))

In [135]:
rmse_test, rmse_train

(327.0425519324707, 141.91248283713122)

In [136]:
len(train.columns) + 1

28

In [137]:
tabla_modelos = pd.concat([tabla_modelos,
           pd.DataFrame(columns=['Modelo', 'RMSE Train', 'RMSE Test', 'Parametros'],
                        data = [['RL', rmse_train, rmse_test, 28]])])

## Lasso

In [138]:
from sklearn.linear_model import LassoCV

In [139]:
lasso_cv = LassoCV(cv=5, random_state=0).fit(train.drop(columns=['Y']), train['Y'])
lasso_cv.score(train.drop(columns=['Y']), train['Y'])

0.9551003390321611

In [140]:
lasso_y_train = lasso_cv.predict(train.drop(columns=['Y']))

In [141]:
lasso_y_test = lasso_cv.predict(test.drop(columns=['Y']))

In [142]:
len(lasso_y_test), len(lasso_y_train)

(18, 229)

In [143]:
residuals_test = test['Y'].values - lasso_y_test
residuals_train = train['Y'].values - lasso_y_train
rmse_train = np.sqrt(np.mean(residuals_train**2))
rmse_test = np.sqrt(np.mean(residuals_test**2))

In [147]:
tabla_modelos = pd.concat([tabla_modelos,
           pd.DataFrame(columns=['Modelo', 'RMSE Train', 'RMSE Test', 'Parametros'],
                        data = [['Lasso', rmse_train, rmse_test, 11]])])

In [148]:
tabla_modelos

,Modelo,RMSE Train,RMSE Test,Parametros
0,Seasonal Naive,316.560422,467.073000,0
0,STL,141.912483,327.042552,0
0,RL,141.912483,327.042552,28
0,Lasso,196.836024,379.268275,28
0,Lasso,196.836024,379.268275,11


In [146]:
lasso_cv.coef_

array([ 0.        ,  0.        ,  0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.        ,  0.        ,  0.39839846,
        0.15337848,  0.02811645, -0.12088341,  0.14149545, -0.06848463,
        0.09755119, -0.15713993,  0.14192894,  0.02762911,  0.        ,
        0.46221524])